<a href="https://colab.research.google.com/github/kangwonlee/py_papago_img/blob/dev/py_papago_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import uuid
import json
import base64
import pathlib



In [ ]:
try:
  import requests_toolbelt
except ImportError:
  !pip install requests_toolbelt
  import requests_toolbelt



In [ ]:
def save_sample_img(write_path:pathlib.Path=pathlib.Path('sample_en.png')):
    url = 'https://github.com/kangwonlee/py_papago_img/blob/dev/sample_en.png'
    response = requests.get(url)
    img = response.content

    write_path.write_bytes(img)



In [ ]:
def translate_img(image_path, src:str, tgt:str, tgt_folder:pathlib.Path, User_client_ID:str, User_client_secret:str):
    data = {
    'source': src,
    'target': tgt,
    'image': (image_path, open(image_path, 'rb'), 'application/octet-stream', {'Content-Transfer-Encoding': 'binary'})
    }
    m = requests_toolbelt.MultipartEncoder(data, boundary=uuid.uuid4())

    headers = {
    "Content-Type": m.content_type,
    "X-NCP-APIGW-API-KEY-ID": User_client_ID,
    "X-NCP-APIGW-API-KEY": User_client_secret
    }

    url = " https://naveropenapi.apigw.ntruss.com/image-to-image/v1/translate"
    res = requests.post(url, headers=headers, data=m.to_string())
    print(res.text)

    # renderedImage -> Image output
    resObj = json.loads(res.text)
    imageStr = resObj.get("data").get("renderedImage")
    imgdata = base64.b64decode(imageStr)

    tgt_path = tgt_folder / image_path.name
    tgt_path.write_bytes(imgdata)

